In [21]:
from typing import List, Dict, Any, Optional, Tuple
import requests
from thx.thx_helper import process_stock_data_all,extract_json_from_js
from thx.thx_tool import BASE_URL
import logging
import pandas as pd
import talib
logger = logging.getLogger(__name__)

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}
def _make_request(url: str, timeout: int = 10, headers: Dict[str, str] = {},**argv) -> Optional[Dict]:
    """发送HTTP请求并提取JSON数据"""
    try:
        print(url)
        common_header = HEADERS.copy()
        if headers:
            common_header.update(headers)
        response = requests.get(url, headers=common_header, timeout=timeout,**argv)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        logger.error(f"请求失败 {url}: {e}")
        return None
url = 'https://q.10jqka.com.cn/zs/detail/code/1A0001/'
url = 'https://q.10jqka.com.cn/zs/detail/code/399001/'
url = 'https://q.10jqka.com.cn/zs/detail/code/399300/'
url = 'https://q.10jqka.com.cn/zs/detail/code/399006/'
response = _make_request(url)

https://q.10jqka.com.cn/zs/detail/code/399006/


In [22]:
from bs4 import BeautifulSoup

def extract_stock_data_hs(response):
    """
    从响应对象中提取股票指数数据
    
    参数:
        response: 包含HTML内容的响应对象，需有text属性
        
    返回:
        dict: 包含股票指数各类数据的字典
    """
    # 创建BeautifulSoup对象
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 定位到board-hq容器并提取数据
    board_hq = soup.find('div', class_='board-hq')
    if not board_hq:
        raise ValueError("未找到class为'board-hq'的div元素")
    
    # 提取指数名称和代码
    h3_tag = board_hq.find('h3')
    index_name = h3_tag.contents[0].strip()
    index_code = h3_tag.find('span').get_text(strip=True)
    
    # 提取当前值
    current_value = board_hq.find('span', class_='board-xj').get_text(strip=True)
    
    # 处理涨跌数据
    zdf_text = board_hq.find('p', class_='board-zdf').get_text(strip=True)
    zdf_parts = zdf_text.split()
    change = zdf_parts[0]
    
    # 构建基础结果字典
    result = {
        '指数名称': index_name,
        '指数代码': index_code,
        '当前指数': float(current_value),
        '涨跌': float(change),
    }
    
    # 提取board-infos中的详细数据
    board_infos = soup.find('div', class_='board-infos')
    if not board_infos:
        raise ValueError("未找到class为'board-infos'的div元素")
    
    dl_tags = board_infos.find_all('dl')
    for dl in dl_tags:
        dt_text = dl.find('dt').get_text(strip=True)
        dd_text = dl.find('dd').get_text(strip=True)
        # 根据内容判断是否转换为数值类型
        if '%' in dd_text:
            result[dt_text] = dd_text
        else:
            result[dt_text] = float(dd_text)
    
    return result

In [23]:
url = 'https://q.10jqka.com.cn/zs/detail/code/1A0001/'
url = 'https://q.10jqka.com.cn/zs/detail/code/399001/'
url = 'https://q.10jqka.com.cn/zs/detail/code/399300/'
url = 'https://q.10jqka.com.cn/zs/detail/code/399006/'

codes  = ('1A0001','399001','399300','399006')
marktet_hq = [ extract_stock_data_hs(_make_request(f'https://q.10jqka.com.cn/zs/detail/code/{code}/')) for code in codes]

https://q.10jqka.com.cn/zs/detail/code/1A0001/
https://q.10jqka.com.cn/zs/detail/code/399001/
https://q.10jqka.com.cn/zs/detail/code/399300/
https://q.10jqka.com.cn/zs/detail/code/399006/


In [24]:
marktet_hq

[{'指数名称': '上证指数',
  '指数代码': '000001',
  '当前指数': 3573.21,
  '涨跌': -42.51,
  '今开': 3604.7,
  '昨收': 3615.72,
  '最低': 3562.61,
  '最高': 3606.37,
  '成交量(万手)': 65450.9,
  '指数涨幅': '-1.18%',
  '成交额(亿)': 8458.93},
 {'指数名称': '深证成指',
  '指数代码': '399001',
  '当前指数': 11009.77,
  '涨跌': -193.26,
  '今开': 11208.46,
  '昨收': 11203.03,
  '最低': 10976.21,
  '最高': 11223.64,
  '成交量(万手)': 74131.1,
  '指数涨幅': '-1.73%',
  '成交额(亿)': 10901.4},
 {'指数名称': '沪深300',
  '指数代码': '399300',
  '当前指数': 4075.59,
  '涨跌': -75.65,
  '今开': 4141.56,
  '昨收': 4151.24,
  '最低': 4063.67,
  '最高': 4141.82,
  '成交量(万手)': 26708.7,
  '指数涨幅': '-1.82%',
  '成交额(亿)': 4921.43},
 {'指数名称': '创业板指',
  '指数代码': '399006',
  '当前指数': 2328.31,
  '涨跌': -39.37,
  '今开': 2382.97,
  '昨收': 2367.68,
  '最低': 2321.72,
  '最高': 2394.22,
  '成交量(万手)': 25898.7,
  '指数涨幅': '-1.66%',
  '成交额(亿)': 5443.91}]